# cards

> Card components for rendering system monitoring dashboards with CPU, memory, disk, network, GPU, process, and temperature information.

In [ ]:
#| default_exp components.cards

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from fasthtml.common import FT

# DaisyUI imports
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title, card_actions
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.data_display.stat import stat, stat_title, stat_value, stat_desc, stats, stats_direction
from cjm_fasthtml_daisyui.components.data_display.table import table, table_modifiers, table_sizes
from cjm_fasthtml_daisyui.components.navigation.tabs import tabs, tab, tab_modifiers, tabs_styles
from cjm_fasthtml_daisyui.components.feedback.progress import progress, progress_colors
from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors
from cjm_fasthtml_daisyui.components.layout.divider import divider
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_daisyui.builders.colors import ColoredUtilityDaisyUI

# Tailwind imports
from cjm_fasthtml_tailwind.utilities.layout import overflow
from cjm_fasthtml_tailwind.utilities.spacing import p, m, space
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, gap, items, justify, grid_display, grid_cols, flex_direction
from cjm_fasthtml_tailwind.utilities.sizing import w, h, min_w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, text_align, font_family, break_all, leading
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.borders import rounded
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_sysmon.core.utils import (
    format_bytes,
    format_bandwidth,
    format_uptime,
    get_temperature_color,
    get_temperature_badge_color,
)
from cjm_fasthtml_sysmon.monitors.system import (
    get_static_system_info, 
)
from cjm_fasthtml_sysmon.components.base import (
    render_process_count,
    render_process_status,
)
from cjm_fasthtml_sysmon.components.common import (
    render_stat_card,
    render_progress_bar,
)
from cjm_fasthtml_sysmon.components.tables import (
    render_cpu_processes_table,
    render_memory_processes_table,
    render_gpu_processes_table
)

# HTML IDs
from cjm_fasthtml_sysmon.core.html_ids import HtmlIds

In [ ]:
#| export
def get_cpu_text_color(
    percent:float  # CPU usage percentage
)-> ColoredUtilityDaisyUI:  # CSS class string for semantic color based on CPU usage
    """Get semantic color based on CPU usage percentage."""
    if percent < 20:
        return text_dui.base_content.opacity(60)  # Idle - subtle
    elif percent < 50:
        return text_dui.success  # Low usage - green
    elif percent < 80:
        return text_dui.warning  # Medium usage - yellow/orange
    else:
        return text_dui.error    # High usage - red

In [ ]:
#| export
def render_cpu_cores_grid(
    cpu_percents:list  # List of CPU usage percentages for each core
)-> FT:  # A Div element containing a responsive grid of CPU core usage
    """Render CPU cores as a responsive grid with color-coded percentages.

    Grid columns are optimized for container width at each breakpoint:
    - Mobile (1 col main): 4 cores wide
    - Small (1 col main): 6 cores wide
    - Medium (2 col main): 4 cores wide (card is ~50% screen)
    - Large (2 col main): 5 cores wide (card is ~50% screen)
    - XL (3 col main): 5 cores wide (card is ~33% screen)
    - 2XL (4 col main): 4-6 cores wide (card is ~25% screen)
    """
    return Div(
        *[Div(
            # Core number (small, subtle)
            Span(f"C{i}", cls=combine_classes(
                font_size.xs,
                text_dui.base_content.opacity(40),  # Very subtle
                font_weight.normal
            )),
            # Percentage value (prominent, colored)
            Div(f"{percent:.0f}%", cls=combine_classes(
                font_size.xs,           # Extra small on mobile
                font_size.sm.sm,        # Small on small screens+
                font_weight.semibold,
                get_cpu_text_color(percent)
            )),
            cls=combine_classes(
                flex_display,
                flex_direction.col,
                items.center,
                justify.center,
                p(1),                   # Smaller padding on mobile
                p(2).sm,                # Normal padding on small+
                bg_dui.base_200,
                rounded.md,
                min_w(10),              # Smaller minimum width on mobile
                min_w(12).sm,           # Normal minimum width on small+
                h(10),                  # Smaller height on mobile
                h(12).sm                # Normal height on small+
            )
        ) for i, percent in enumerate(cpu_percents)],
        cls=combine_classes(
            grid_display,
            # Responsive grid columns - adjusted for container context
            grid_cols(4),       # Mobile (1 col main): 4 cores wide
            grid_cols(6).sm,    # Small (1 col main): 6 cores wide
            grid_cols(4).md,    # Medium (2 col main): 4 cores per row (good for ~50% width)
            grid_cols(5).lg,    # Large (2 col main): 5 cores per row (good for ~50% width)
            grid_cols(5).xl,    # XL (3 col main): 5 cores per row (good for ~33% width)
            grid_cols(6)._2xl,  # 2XL (4 col main): 6 cores per row (good for ~25% width)
            gap(1),             # Small gap between items
            w.full
        )
    )

In [ ]:
#| export
def render_os_info_card()-> FT:  # A Div element containing the OS information card
    """Render the OS information card."""
    info = get_static_system_info()

    return Div(
        Div(
            H3("Operating System", cls=combine_classes(card_title, text_dui.base_content)),
            cls=str(m.b(4))
        ),
        Div(
            render_stat_card("System", f"{info['os']} {info['os_release']}", info['architecture']),
            render_stat_card("Hostname", info['hostname'], f"Python {info['python_version']}"),
            render_stat_card("Boot Time", info['boot_time'], f"Uptime: {format_uptime(info['boot_time'])}"),
            render_stat_card("CPU Cores", f"{info['cpu_count']} Physical", f"{info['cpu_count_logical']} Logical"),
            cls=combine_classes(stats, stats_direction.vertical, bg_dui.base_200, rounded.lg, p(4), overflow.x.auto, w.full)
        ),
        cls=str(card_body)
    )

In [ ]:
render_os_info_card()

```html
<div class="card-body">
  <div class="mb-4">
    <h3 class="card-title text-base-content">Operating System</h3>
  </div>
  <div class="stats stats-vertical bg-base-200 rounded-lg p-4 overflow-x-auto w-full">
    <div class="stat">
      <div class="stat-title text-base-content">System</div>
      <div class="stat-value">Linux 6.14.0-33-generic</div>
      <div class="stat-desc">x86_64</div>
    </div>
    <div class="stat">
      <div class="stat-title text-base-content">Hostname</div>
      <div class="stat-value">innom-dt</div>
      <div class="stat-desc">Python 3.11.13</div>
    </div>
    <div class="stat">
      <div class="stat-title text-base-content">Boot Time</div>
      <div class="stat-value">2025-10-01 10:16:11</div>
      <div class="stat-desc">Uptime: 7h 16m</div>
    </div>
    <div class="stat">
      <div class="stat-title text-base-content">CPU Cores</div>
      <div class="stat-value">16 Physical</div>
      <div class="stat-desc">32 Logical</div>
    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_cpu_card(
    cpu_info:dict  # Dictionary containing CPU usage information
)-> FT:  # A Div element containing the CPU usage card
    """Render the CPU usage card."""
    return Div(
        Div(
            H3("CPU Usage", cls=combine_classes(card_title, text_dui.base_content)),
            Span(
                f"{cpu_info['percent']:.1f}%",
                cls=combine_classes(
                    badge,
                    badge_colors.primary if cpu_info['percent'] < 80 else badge_colors.error,
                    badge_sizes.lg
                )
            ),
            cls=combine_classes(flex_display, justify.between, items.center, m.b(4))
        ),

        # Overall CPU usage
        Div(
            render_progress_bar(cpu_info['percent'], label="Overall Usage"),
            cls=str(m.b(4))
        ),

        # CPU Frequency
        Div(
            P("CPU Frequency", cls=combine_classes(font_size.sm, font_weight.medium, m.b(2))),
            Div(
                Span(f"Current: {cpu_info['frequency_current']:.0f} MHz",
                     cls=combine_classes(text_dui.primary, font_size.sm)),
                Span(f"Min: {cpu_info['frequency_min']:.0f} MHz",
                     cls=combine_classes(text_dui.base_content, font_size.xs)),
                Span(f"Max: {cpu_info['frequency_max']:.0f} MHz",
                     cls=combine_classes(text_dui.base_content, font_size.xs)),
                cls=combine_classes(flex_display, justify.between, gap(2))
            ),
            cls=str(m.b(4))
        ),

        # Per-core usage - now handles any number of cores efficiently
        Div(
            P("Per Core Usage", cls=combine_classes(font_size.sm, font_weight.medium, m.b(2))),
            render_cpu_cores_grid(cpu_info['percent_per_core']),
            cls=str(m.t(4))
        ) if cpu_info['percent_per_core'] else None,

        cls=str(card_body),
        id=HtmlIds.CPU_CARD_BODY
    )

In [ ]:
from cjm_fasthtml_sysmon.monitors.cpu import get_cpu_info

cpu_info = get_cpu_info()
render_cpu_card(cpu_info=cpu_info)

```html
<div id="cpu-card-body" class="card-body">
  <div class="flex justify-between items-center mb-4">
    <h3 class="card-title text-base-content">CPU Usage</h3>
<span class="badge badge-primary badge-lg">3.1%</span>  </div>
  <div class="mb-4">
    <div>
      <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">Overall Usage</span><span class="text-xs text-base-content">3.1%</span>      </div>
<progress value="3.1" max="100" class="progress progress-success w-full"></progress>    </div>
  </div>
  <div class="mb-4">
    <p class="text-sm font-medium mb-2">CPU Frequency</p>
    <div class="flex justify-between gap-2">
<span class="text-primary text-sm">Current: 4110 MHz</span><span class="text-base-content text-xs">Min: 2982 MHz</span><span class="text-base-content text-xs">Max: 5756 MHz</span>    </div>
  </div>
  <div class="mt-4">
    <p class="text-sm font-medium mb-2">Per Core Usage</p>
    <div class="grid grid-cols-4 sm:grid-cols-6 md:grid-cols-4 lg:grid-cols-5 xl:grid-cols-5 2xl:grid-cols-6 gap-1 w-full">
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C0</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C1</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C2</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C3</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C4</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C5</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C6</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C7</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C8</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C9</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C10</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C11</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C12</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C13</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C14</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C15</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C16</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C17</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C18</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C19</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C20</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">9%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C21</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C22</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C23</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C24</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C25</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C26</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C27</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C28</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C29</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C30</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
      <div class="flex flex-col items-center justify-center p-1 sm:p-2 bg-base-200 rounded-md min-w-10 sm:min-w-12 h-10 sm:h-12">
<span class="text-xs text-base-content/40 font-normal">C31</span>        <div class="text-xs sm:text-sm font-semibold text-base-content/60">0%</div>
      </div>
    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_memory_card(
    mem_info:dict  # Dictionary containing memory usage information
)-> FT:  # A Div element containing the memory usage card
    """Render the memory usage card."""
    return Div(
        Div(
            H3("Memory Usage", cls=combine_classes(card_title, text_dui.base_content)),
            Span(
                f"{mem_info['percent']:.1f}%",
                cls=combine_classes(
                    badge,
                    badge_colors.primary if mem_info['percent'] < 80 else badge_colors.error,
                    badge_sizes.lg
                )
            ),
            cls=combine_classes(flex_display, justify.between, items.center, m.b(4))
        ),

        # RAM Usage
        Div(
            P("RAM", cls=combine_classes(font_size.sm, font_weight.medium, m.b(2))),
            render_progress_bar(mem_info['percent'],
                              label=f"{format_bytes(mem_info['used'])} / {format_bytes(mem_info['total'])}"),
            P(f"Available: {format_bytes(mem_info['available'])}",
              cls=combine_classes(font_size.xs, text_dui.base_content, m.t(1))),
            cls=str(m.b(4))
        ),

        # Swap Usage
        Div(
            P("Swap", cls=combine_classes(font_size.sm, font_weight.medium, m.b(2))),
            render_progress_bar(mem_info['swap_percent'],
                              label=f"{format_bytes(mem_info['swap_used'])} / {format_bytes(mem_info['swap_total'])}"),
            cls=str(m.t(4))
        ) if mem_info['swap_total'] > 0 else None,

        cls=str(card_body),
        id=HtmlIds.MEMORY_CARD_BODY
    )

In [ ]:
from cjm_fasthtml_sysmon.monitors.memory import get_memory_info

mem_info = get_memory_info()
render_memory_card(mem_info=mem_info)

```html
<div id="memory-card-body" class="card-body">
  <div class="flex justify-between items-center mb-4">
    <h3 class="card-title text-base-content">Memory Usage</h3>
<span class="badge badge-primary badge-lg">15.0%</span>  </div>
  <div class="mb-4">
    <p class="text-sm font-medium mb-2">RAM</p>
    <div>
      <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">13.8 GB / 91.9 GB</span><span class="text-xs text-base-content">15.0%</span>      </div>
<progress value="15.0" max="100" class="progress progress-success w-full"></progress>    </div>
    <p class="text-xs text-base-content mt-1">Available: 78.1 GB</p>
  </div>
  <div class="mt-4">
    <p class="text-sm font-medium mb-2">Swap</p>
    <div>
      <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">0.0 B / 72.0 GB</span><span class="text-xs text-base-content">0.0%</span>      </div>
<progress value="0.0" max="100" class="progress progress-success w-full"></progress>    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_disk_entries(
    disk_info:list  # List of dictionaries containing disk information
)-> FT:  # A Div element containing disk entries
    """Render just the disk entries section."""
    return Div(
        *[Div(
            Div(
                P(disk['device'], cls=combine_classes(font_size.sm, font_weight.medium)),
                P(f"{disk['mountpoint']} ({disk['fstype']})",
                  cls=combine_classes(font_size.xs, text_dui.base_content)),
                cls=str(m.b(2))
            ),
            render_progress_bar(disk['percent'],
                              label=f"{format_bytes(disk['used'])} / {format_bytes(disk['total'])}"),
            P(f"Free: {format_bytes(disk['free'])}",
              cls=combine_classes(font_size.xs, text_dui.base_content, m.t(1))),
            cls=combine_classes(p(3), bg_dui.base_200, rounded.lg, m.b(3))
        ) for disk in disk_info[:5]],
        cls="",
        id=HtmlIds.DISK_ENTRIES
    )

In [ ]:
#| export
def render_disk_card(
    disk_info:list  # List of dictionaries containing disk usage information
)-> FT:  # A Div element containing the disk usage card
    """Render the disk usage card using helper functions."""
    return Div(
        Div(
            H3("Disk Usage", cls=combine_classes(card_title, text_dui.base_content)),
            cls=str(m.b(4))
        ),

        # Disk entries - render using helper
        render_disk_entries(disk_info),

        cls=str(card_body),
        id=HtmlIds.DISK_CARD_BODY
    )

In [ ]:
from cjm_fasthtml_sysmon.monitors.disk import get_disk_info

disk_info = get_disk_info()
render_disk_card(disk_info=disk_info)

```html
<div id="disk-card-body" class="card-body">
  <div class="mb-4">
    <h3 class="card-title text-base-content">Disk Usage</h3>
  </div>
  <div id="disk-entries">
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">/dev/nvme0n1p6</p>
        <p class="text-xs text-base-content">/ (ext4)</p>
      </div>
      <div>
        <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">285.8 GB / 491.1 GB</span><span class="text-xs text-base-content">61.3%</span>        </div>
<progress value="61.3" max="100" class="progress progress-warning w-full"></progress>      </div>
      <p class="text-xs text-base-content mt-1">Free: 180.3 GB</p>
    </div>
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">/dev/loop2</p>
        <p class="text-xs text-base-content">/snap/bare/5 (squashfs)</p>
      </div>
      <div>
        <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">128.0 KB / 128.0 KB</span><span class="text-xs text-base-content">100.0%</span>        </div>
<progress value="100.0" max="100" class="progress progress-error w-full"></progress>      </div>
      <p class="text-xs text-base-content mt-1">Free: 0.0 B</p>
    </div>
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">/dev/loop0</p>
        <p class="text-xs text-base-content">/snap/aws-cli/1604 (squashfs)</p>
      </div>
      <div>
        <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">52.1 MB / 52.1 MB</span><span class="text-xs text-base-content">100.0%</span>        </div>
<progress value="100.0" max="100" class="progress progress-error w-full"></progress>      </div>
      <p class="text-xs text-base-content mt-1">Free: 0.0 B</p>
    </div>
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">/dev/loop1</p>
        <p class="text-xs text-base-content">/snap/aws-cli/1609 (squashfs)</p>
      </div>
      <div>
        <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">52.1 MB / 52.1 MB</span><span class="text-xs text-base-content">100.0%</span>        </div>
<progress value="100.0" max="100" class="progress progress-error w-full"></progress>      </div>
      <p class="text-xs text-base-content mt-1">Free: 0.0 B</p>
    </div>
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">/dev/loop4</p>
        <p class="text-xs text-base-content">/snap/cmake/1481 (squashfs)</p>
      </div>
      <div>
        <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">52.0 MB / 52.0 MB</span><span class="text-xs text-base-content">100.0%</span>        </div>
<progress value="100.0" max="100" class="progress progress-error w-full"></progress>      </div>
      <p class="text-xs text-base-content mt-1">Free: 0.0 B</p>
    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_network_interfaces(
    net_info:dict  # Dictionary containing network information
)-> FT:  # A Div element containing network interfaces
    """Render just the network interfaces section."""
    interfaces = net_info['interfaces']

    return Div(
        *[Div(
            # Interface header
            Div(
                P(interface['name'], cls=combine_classes(font_size.sm, font_weight.medium)),
                P(', '.join(interface['ip_addresses']) if interface['ip_addresses'] else 'No IP',
                  cls=combine_classes(font_size.xs, text_dui.base_content)),
                cls=str(m.b(2))
            ),

            # Bandwidth meters
            Div(
                # Upload speed
                Label(
                    Label(
                        Span("↑ Upload", cls=combine_classes(font_size.xs, text_dui.base_content)),
                        Span(format_bandwidth(interface['bytes_sent_per_sec']),
                             cls=combine_classes(font_size.xs, text_dui.info, font_weight.medium)),
                        cls=combine_classes(flex_display, justify.between)
                    ),
                    Progress(
                        value=str(min(100, interface['bytes_sent_per_sec'] / 1024 / 1024 * 10)),
                        max="100",
                        cls=combine_classes(progress, progress_colors.info, w.full, h(1))
                    ),
                    cls=str(m.b(2))
                ),

                # Download speed
                Label(
                    Label(
                        Span("↓ Download", cls=combine_classes(font_size.xs, text_dui.base_content)),
                        Span(format_bandwidth(interface['bytes_recv_per_sec']),
                             cls=combine_classes(font_size.xs, text_dui.success, font_weight.medium)),
                        cls=combine_classes(flex_display, justify.between)
                    ),
                    Progress(
                        value=str(min(100, interface['bytes_recv_per_sec'] / 1024 / 1024 * 10)),
                        max="100",
                        cls=combine_classes(progress, progress_colors.success, w.full, h(1))
                    ),
                    cls=str(m.b(2))
                ),

                # Statistics
                Label(
                    Span(f"Total: ↑{format_bytes(interface['bytes_sent'])} ↓{format_bytes(interface['bytes_recv'])}",
                         cls=combine_classes(font_size.xs, text_dui.base_content)),
                    cls=str(m.t(1))
                ),
            ),

            cls=combine_classes(p(3), bg_dui.base_200, rounded.lg, m.b(3))
        ) for interface in interfaces[:3]],
        cls="",
        id=HtmlIds.NETWORK_INTERFACES
    )

In [ ]:
#| export
def render_network_connections(
    net_info:dict  # Dictionary containing network information
)-> FT:  # A Div element containing connection statistics
    """Render just the network connections section."""
    connections = net_info['connections']

    return Div(
        P("Connections", cls=combine_classes(font_size.sm, font_weight.medium, m.b(2))),
        Div(
            render_stat_card("Total", str(connections['total'])),
            render_stat_card("Established", str(connections['established'])),
            render_stat_card("Listening", str(connections['listen'])),
            render_stat_card("Time Wait", str(connections['time_wait'])),
            cls=combine_classes(stats, bg_dui.base_200, rounded.lg, p(2), font_size.xs, overflow.x.auto, w.full)
        ),
        cls=str(m.t(3)),
        id=HtmlIds.NETWORK_CONNECTIONS
    )

In [ ]:
#| export
def render_network_card(
    net_info:dict  # Dictionary containing network interface and connection information
)-> FT:  # A Div element containing the network monitoring card
    """Render the network monitoring card using helper functions."""
    interfaces = net_info['interfaces']

    if not interfaces:
        return Div(
            Div(
                H3("Network", cls=combine_classes(card_title, text_dui.base_content)),
                cls=str(m.b(4))
            ),
            Div(
                "No active network interfaces detected",
                cls=combine_classes(alert, alert_colors.info)
            ),
            cls=str(card_body)
        )

    return Div(
        Div(
            H3("Network", cls=combine_classes(card_title, text_dui.base_content)),
            Span(
                f"{len(interfaces)} Active",
                cls=combine_classes(badge, badge_colors.info, badge_sizes.lg)
            ),
            cls=combine_classes(flex_display, justify.between, items.center, m.b(4))
        ),

        # Network interfaces - render using helper
        render_network_interfaces(net_info),

        # Connection statistics - render using helper
        render_network_connections(net_info),

        cls=str(card_body),
        id=HtmlIds.NETWORK_CARD_BODY
    )

In [ ]:
from cjm_fasthtml_sysmon.monitors.network import get_network_info

net_info = get_network_info()
render_network_card(net_info=net_info)

```html
<div id="network-card-body" class="card-body">
  <div class="flex justify-between items-center mb-4">
    <h3 class="card-title text-base-content">Network</h3>
<span class="badge badge-info badge-lg">4 Active</span>  </div>
  <div id="network-interfaces">
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">enp7s0</p>
        <p class="text-xs text-base-content">192.168.2.41</p>
      </div>
      <div>
<label class="mb-2"><label class="flex justify-between"><span class="text-xs text-base-content">↑ Upload</span><span class="text-xs text-info font-medium">0 B/s</span></label><progress value="0.0" max="100" class="progress progress-info w-full h-1"></progress></label><label class="mb-2"><label class="flex justify-between"><span class="text-xs text-base-content">↓ Download</span><span class="text-xs text-success font-medium">0 B/s</span></label><progress value="0.0" max="100" class="progress progress-success w-full h-1"></progress></label><label class="mt-1"><span class="text-xs text-base-content">Total: ↑402.9 MB ↓5.0 GB</span></label>      </div>
    </div>
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">enp8s0</p>
        <p class="text-xs text-base-content">No IP</p>
      </div>
      <div>
<label class="mb-2"><label class="flex justify-between"><span class="text-xs text-base-content">↑ Upload</span><span class="text-xs text-info font-medium">0 B/s</span></label><progress value="0.0" max="100" class="progress progress-info w-full h-1"></progress></label><label class="mb-2"><label class="flex justify-between"><span class="text-xs text-base-content">↓ Download</span><span class="text-xs text-success font-medium">0 B/s</span></label><progress value="0.0" max="100" class="progress progress-success w-full h-1"></progress></label><label class="mt-1"><span class="text-xs text-base-content">Total: ↑0.0 B ↓0.0 B</span></label>      </div>
    </div>
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <div class="mb-2">
        <p class="text-sm font-medium">wlp15s0</p>
        <p class="text-xs text-base-content">No IP</p>
      </div>
      <div>
<label class="mb-2"><label class="flex justify-between"><span class="text-xs text-base-content">↑ Upload</span><span class="text-xs text-info font-medium">0 B/s</span></label><progress value="0.0" max="100" class="progress progress-info w-full h-1"></progress></label><label class="mb-2"><label class="flex justify-between"><span class="text-xs text-base-content">↓ Download</span><span class="text-xs text-success font-medium">0 B/s</span></label><progress value="0.0" max="100" class="progress progress-success w-full h-1"></progress></label><label class="mt-1"><span class="text-xs text-base-content">Total: ↑0.0 B ↓0.0 B</span></label>      </div>
    </div>
  </div>
  <div id="network-connections" class="mt-3">
    <p class="text-sm font-medium mb-2">Connections</p>
    <div class="stats bg-base-200 rounded-lg p-2 text-xs overflow-x-auto w-full">
      <div class="stat">
        <div class="stat-title text-base-content">Total</div>
        <div class="stat-value">223</div>
      </div>
      <div class="stat">
        <div class="stat-title text-base-content">Established</div>
        <div class="stat-value">102</div>
      </div>
      <div class="stat">
        <div class="stat-title text-base-content">Listening</div>
        <div class="stat-value">39</div>
      </div>
      <div class="stat">
        <div class="stat-title text-base-content">Time Wait</div>
        <div class="stat-value">72</div>
      </div>
    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_process_card(
    proc_info:dict  # Dictionary containing process information and statistics
)-> FT:  # A Div element containing the process monitoring card
    """Render the process monitoring card."""
    return Div(
        # Header with process count
        Div(
            H3("Process Monitor", cls=combine_classes(card_title, text_dui.base_content)),
            render_process_count(proc_info['total']),
            cls=combine_classes(flex_display, justify.between, items.center, m.b(4))
        ),

        # Process status summary
        render_process_status(proc_info['status_counts']),

        # Tabs for CPU vs Memory view
        Div(
            # Tab buttons
            Div(
                Button("Top CPU",
                       cls=combine_classes(tab, tab_modifiers.active),
                       id=HtmlIds.CPU_TAB,
                       onclick=f"document.getElementById('{HtmlIds.CPU_PROCESSES}').style.display='block'; document.getElementById('{HtmlIds.MEMORY_PROCESSES}').style.display='none'; this.classList.add('tab-active'); document.getElementById('{HtmlIds.MEM_TAB}').classList.remove('tab-active')"),
                Button("Top Memory",
                       cls=str(tab),
                       id=HtmlIds.MEM_TAB,
                       onclick=f"document.getElementById('{HtmlIds.MEMORY_PROCESSES}').style.display='block'; document.getElementById('{HtmlIds.CPU_PROCESSES}').style.display='none'; this.classList.add('tab-active'); document.getElementById('{HtmlIds.CPU_TAB}').classList.remove('tab-active')"),
                role="tablist",
                cls=combine_classes(tabs, tabs_styles.box)
            ),
            cls=str(m.b(4))
        ),

        # Top CPU processes table
        Div(
            render_cpu_processes_table(proc_info['top_cpu']),
            id=HtmlIds.CPU_PROCESSES,
            style="display: block;",
            cls=combine_classes(overflow.x.auto)
        ),

        # Top Memory processes table
        Div(
            render_memory_processes_table(proc_info['top_memory']),
            id=HtmlIds.MEMORY_PROCESSES,
            style="display: none;",
            cls=combine_classes(overflow.x.auto)
        ),

        cls=str(card_body),
        id=HtmlIds.PROCESS_CARD_BODY
    )

In [ ]:
from cjm_fasthtml_sysmon.monitors.processes import get_process_info

proc_info = get_process_info()
render_process_card(proc_info=proc_info)

```html
<div id="process-card-body" class="card-body">
  <div class="flex justify-between items-center mb-4">
    <h3 class="card-title text-base-content">Process Monitor</h3>
<span id="process-count" class="badge badge-primary badge-lg">206 processes</span>  </div>
  <div id="process-status" class="flex flex-wrap gap-1 mb-4">
<span class="badge badge-neutral badge-sm mr-2">sleeping: 205</span><span class="badge badge-info badge-sm mr-2">running: 1</span>  </div>
  <div class="mb-4">
    <div role="tablist" class="tabs tabs-box">
<button onclick="document.getElementById('cpu-processes').style.display='block'; document.getElementById('memory-processes').style.display='none'; this.classList.add('tab-active'); document.getElementById('mem-tab').classList.remove('tab-active')" id="cpu-tab" class="tab tab-active" name="cpu-tab">Top CPU</button><button onclick="document.getElementById('memory-processes').style.display='block'; document.getElementById('cpu-processes').style.display='none'; this.classList.add('tab-active'); document.getElementById('cpu-tab').classList.remove('tab-active')" id="mem-tab" class="tab" name="mem-tab">Top Memory</button>    </div>
  </div>
  <div id="cpu-processes" class="overflow-x-auto" style="display: block;">
    <div id="cpu-processes-table">
      <table class="table table-zebra table-xs w-full">
        <thead>
          <tr>
            <th class="text-xs w-16">PID</th>
            <th class="text-xs">Name</th>
            <th class="text-xs w-20">CPU %</th>
            <th class="text-xs w-24">Memory</th>
            <th class="text-xs">User</th>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td class="text-xs">1</td>
            <td class="text-xs font-medium">systemd</td>
            <td>
<label class="badge badge-info badge-sm">0.0%</label>            </td>
            <td class="text-xs">14 MB</td>
            <td class="text-xs">root</td>
          </tr>
          <tr>
            <td class="text-xs">621</td>
            <td class="text-xs font-medium">systemd-journald</td>
            <td>
<label class="badge badge-info badge-sm">0.0%</label>            </td>
            <td class="text-xs">17 MB</td>
            <td class="text-xs">root</td>
          </tr>
          <tr>
            <td class="text-xs">696</td>
            <td class="text-xs font-medium">systemd-udevd</td>
            <td>
<label class="badge badge-info badge-sm">0.0%</label>            </td>
            <td class="text-xs">7 MB</td>
            <td class="text-xs">root</td>
          </tr>
          <tr>
            <td class="text-xs">1175</td>
            <td class="text-xs font-medium">systemd-oomd</td>
            <td>
<label class="badge badge-info badge-sm">0.0%</label>            </td>
            <td class="text-xs">7 MB</td>
            <td class="text-xs">systemd-oom</td>
          </tr>
          <tr>
            <td class="text-xs">1191</td>
            <td class="text-xs font-medium">systemd-resolved</td>
            <td>
<label class="badge badge-info badge-sm">0.0%</label>            </td>
            <td class="text-xs">12 MB</td>
            <td class="text-xs">systemd-resolve</td>
          </tr>
        </tbody>
      </table>
    </div>
  </div>
  <div id="memory-processes" class="overflow-x-auto" style="display: none;">
    <div id="memory-processes-table">
      <table class="table table-zebra table-xs w-full">
        <thead>
          <tr>
            <th class="text-xs w-16">PID</th>
            <th class="text-xs">Name</th>
            <th class="text-xs w-20">Memory %</th>
            <th class="text-xs w-24">Memory</th>
            <th class="text-xs">User</th>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td class="text-xs">14235</td>
            <td class="text-xs font-medium">firefox</td>
            <td>
<label class="badge badge-info badge-sm">2.4%</label>            </td>
            <td class="text-xs">2222 MB</td>
            <td class="text-xs">innom-dt</td>
          </tr>
          <tr>
            <td class="text-xs">11539</td>
            <td class="text-xs font-medium">nautilus</td>
            <td>
<label class="badge badge-info badge-sm">1.5%</label>            </td>
            <td class="text-xs">1417 MB</td>
            <td class="text-xs">innom-dt</td>
          </tr>
          <tr>
            <td class="text-xs">15126</td>
            <td class="text-xs font-medium">Isolated Web Co</td>
            <td>
<label class="badge badge-info badge-sm">0.9%</label>            </td>
            <td class="text-xs">887 MB</td>
            <td class="text-xs">innom-dt</td>
          </tr>
          <tr>
            <td class="text-xs">12565</td>
            <td class="text-xs font-medium">code</td>
            <td>
<label class="badge badge-info badge-sm">0.8%</label>            </td>
            <td class="text-xs">744 MB</td>
            <td class="text-xs">innom-dt</td>
          </tr>
          <tr>
            <td class="text-xs">210843</td>
            <td class="text-xs font-medium">code</td>
            <td>
<label class="badge badge-info badge-sm">0.7%</label>            </td>
            <td class="text-xs">679 MB</td>
            <td class="text-xs">innom-dt</td>
          </tr>
        </tbody>
      </table>
    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_gpu_metrics(
    gpu_info:dict  # Dictionary containing GPU information
)-> FT:  # A Div element containing GPU metrics section
    """Render just the GPU metrics section (utilization, memory, temp, power)."""
    if not gpu_info['available']:
        return None

    return Div(
        *[Div(
            P(details['name'], cls=combine_classes(font_size.sm, font_weight.medium, m.b(2))),

            # Main metrics in grid
            Div(
                # GPU Utilization
                Label(
                    P("GPU Utilization", cls=combine_classes(font_size.xs, text_dui.base_content)),
                    render_progress_bar(details['utilization']),
                    cls=str(m.b(3))
                ),

                # GPU Memory
                Label(
                    P("Memory", cls=combine_classes(font_size.xs, text_dui.base_content)),
                    render_progress_bar(
                        (details['memory_used'] / details['memory_total']) * 100 if details['memory_total'] > 0 else 0,
                        label=f"{details['memory_used']} MB / {details['memory_total']} MB"
                    ),
                    cls=str(m.b(3))
                ),

                # Temperature (if available)
                Label(
                    P("Temperature", cls=combine_classes(font_size.xs, text_dui.base_content)),
                    Label(
                        Span(
                            f"{details.get('temperature', 'N/A')}°C" if details.get('temperature') else "N/A",
                            cls=combine_classes(
                                font_weight.medium,
                                get_temperature_color(details.get('temperature', 0), 80, 90) if details.get('temperature') else text_dui.base_content
                            )
                        ),
                        cls=str(m.t(1))
                    ),
                    cls=str(m.b(3))
                ) if details.get('temperature') is not None else None,

                # Power Usage (if available)
                Label(
                    P("Power", cls=combine_classes(font_size.xs, text_dui.base_content)),
                    Label(
                        Span(
                            f"{details.get('power_usage', 0):.1f}W / {details.get('power_limit', 0):.1f}W"
                            if details.get('power_usage') is not None else "N/A",
                            cls=combine_classes(font_size.sm, text_dui.base_content)
                        ),
                        render_progress_bar(
                            (details.get('power_usage', 0) / details.get('power_limit', 1)) * 100
                            if details.get('power_limit') and details.get('power_limit') > 0 else 0,
                            label=None
                        ) if details.get('power_usage') is not None and details.get('power_limit') else None,
                        cls=""
                    ),
                    cls=str(m.b(3))
                ) if details.get('power_usage') is not None else None,

                # Additional metrics in a row
                Label(
                    # Fan Speed
                    Span(
                        f"Fan: {details.get('fan_speed', 'N/A')}%" if details.get('fan_speed') is not None else "",
                        cls=combine_classes(font_size.xs, text_dui.base_content)
                    ) if details.get('fan_speed') is not None else None,

                    # Encoder/Decoder utilization
                    Span(
                        f"Enc: {details.get('encoder_utilization', 0)}%",
                        cls=combine_classes(font_size.xs, text_dui.base_content, m.l(3))
                    ) if details.get('encoder_utilization') is not None else None,

                    Span(
                        f"Dec: {details.get('decoder_utilization', 0)}%",
                        cls=combine_classes(font_size.xs, text_dui.base_content, m.l(3))
                    ) if details.get('decoder_utilization') is not None else None,

                    # Process count
                    Span(
                        f"Processes: {details.get('compute_processes', 0)}",
                        cls=combine_classes(font_size.xs, text_dui.base_content, m.l(3))
                    ) if details.get('compute_processes') is not None else None,

                    cls=combine_classes(flex_display, items.center)
                ) if any([details.get('fan_speed'), details.get('encoder_utilization'),
                         details.get('decoder_utilization'), details.get('compute_processes')]) else None,

                cls=""
            ),

            cls=combine_classes(p(3), bg_dui.base_200, rounded.lg, m.b(3))
        ) for gpu_id, details in gpu_info['details'].items()],
        cls="",
        id=HtmlIds.GPU_METRICS
    )

In [ ]:
#| export
def render_gpu_processes_section(
    gpu_info:dict  # Dictionary containing GPU information
)-> FT:  # A Div element containing GPU processes section
    """Render just the GPU processes section."""
    if not gpu_info['available'] or gpu_info.get('processes') is None:
        return None

    return Div(
        Div(cls=combine_classes(divider, m.y(3))),
        P("GPU Processes", cls=combine_classes(font_size.sm, font_weight.semibold, m.b(3), text_dui.base_content)),

        # Process table - now uses the extracted table function
        render_gpu_processes_table(gpu_info.get('processes', [])),
        
        cls="",
        id=HtmlIds.GPU_PROCESSES_SECTION
    )

In [ ]:
#| export
def render_gpu_card(
    gpu_info:dict  # Dictionary containing GPU information and statistics
)-> FT:  # A Div element containing the GPU information card
    """Render the GPU information card using helper functions."""
    if not gpu_info['available']:
        return Div(
            Div(
                H3("GPU Information", cls=combine_classes(card_title, text_dui.base_content)),
                cls=str(m.b(4))
            ),
            Div(
                "No GPU detected or GPU monitoring not available",
                cls=combine_classes(alert, alert_colors.info)
            ),
            cls=str(card_body)
        )

    return Div(
        Div(
            H3("GPU Information", cls=combine_classes(card_title, text_dui.base_content)),
            Span(
                gpu_info['type'],
                cls=combine_classes(badge, badge_colors.success, badge_sizes.lg)
            ),
            cls=combine_classes(flex_display, justify.between, items.center, m.b(4))
        ),

        # GPU Metrics Section - render using helper
        render_gpu_metrics(gpu_info),

        # GPU Processes Section - render using helper
        render_gpu_processes_section(gpu_info),

        cls=str(card_body),
        id=HtmlIds.GPU_CARD_BODY
    )

In [ ]:
#| eval: false
from cjm_fasthtml_sysmon.monitors.gpu import get_gpu_info

gpu_info = get_gpu_info()
render_gpu_card(gpu_info=gpu_info)

```html
<div id="gpu-card-body" class="card-body">
  <div class="flex justify-between items-center mb-4">
    <h3 class="card-title text-base-content">GPU Information</h3>
<span class="badge badge-success badge-lg">NVIDIA</span>  </div>
  <div id="gpu-metrics">
    <div class="p-3 bg-base-200 rounded-lg mb-3">
      <p class="text-sm font-medium mb-2">NVIDIA GeForce RTX 4090</p>
      <div>
<label class="mb-3">          <p class="text-xs text-base-content">GPU Utilization</p>
          <div>
<progress value="8" max="100" class="progress progress-success w-full"></progress>          </div>
</label><label class="mb-3">          <p class="text-xs text-base-content">Memory</p>
          <div>
            <div class="justify-between mb-1 flex">
<span class="text-xs text-base-content">1350 MB / 24564 MB</span><span class="text-xs text-base-content">5.5%</span>            </div>
<progress value="5.495847581827064" max="100" class="progress progress-success w-full"></progress>          </div>
</label><label class="mb-3">          <p class="text-xs text-base-content">Temperature</p>
<label class="mt-1"><span class="font-medium text-success">47°C</span></label></label><label class="mb-3">          <p class="text-xs text-base-content">Power</p>
<label><span class="text-sm text-base-content">22.5W / 450.0W</span>            <div>
<progress value="4.998666666666667" max="100" class="progress progress-success w-full"></progress>            </div>
</label></label><label class="flex items-center"><span class="text-xs text-base-content">Fan: 0%</span><span class="text-xs text-base-content ml-3">Enc: 0%</span><span class="text-xs text-base-content ml-3">Dec: 0%</span><span class="text-xs text-base-content ml-3">Processes: 9</span></label>      </div>
    </div>
  </div>
  <div id="gpu-processes-section">
    <div class="divider my-3"></div>
    <p class="text-sm font-semibold mb-3 text-base-content">GPU Processes</p>
    <div id="gpu-processes-table" class="overflow-x-auto bg-base-200 rounded-lg p-2">
      <table class="table table-xs w-full">
        <thead>
          <tr>
            <th class="text-xs font-medium text-base-content">PID</th>
            <th class="text-xs font-medium text-base-content">Process</th>
            <th class="text-xs font-medium text-base-content">GPU Memory</th>
            <th class="text-xs font-medium text-base-content">GPU Usage</th>
            <th class="text-xs font-medium text-base-content">Device</th>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td class="text-xs text-base-content">5223</td>
            <td class="text-xs font-medium">/usr/lib/xorg/Xorg vt2 -displayfd 3 -auth /run/user/1000/gdm/Xauthority -nolisten tcp -background none -noreset -keeptty -novtswitch -verbose 3</td>
            <td>
<span class="badge badge-primary badge-xs">640 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">14235</td>
            <td class="text-xs font-medium">/snap/firefox/6933/usr/lib/firefox/firefox</td>
            <td>
<span class="badge badge-primary badge-xs">314 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">5452</td>
            <td class="text-xs font-medium">/usr/bin/gnome-shell</td>
            <td>
<span class="badge badge-primary badge-xs">63 MB</span>            </td>
            <td class="text-xs text-success">18%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">11782</td>
            <td class="text-xs font-medium">/proc/self/exe --type=gpu-process --disable-gpu-sandbox --no-sandbox --enable-crash-reporter=94bcc087-1c49-442f-93c6-80a5155613d1,no_channel --user-data-dir=/home/innom-dt/.config/Code --gpu-preferences=UAAAAAAAAAAgAAAEAAAAAAAAAAAAAAAAAABgAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAABAAAAAAAAAAEAAAAAAAAAAIAAAAAAAAAAgAAAAAAAAA --shared-files --field-trial-handle=3,i,11272903268968390665,6950229763176126348,262144 --enable-features=DocumentPolicyIncludeJSCallStacksInCrashReports,EarlyEstablishGpuChannel,EstablishGpuChannelAsync --disable-features=CalculateNativeWinOcclusion,FontationsLinuxSystemFonts,ScreenAIOCREnabled,SpareRendererForSitePerProcess --variations-seed-version</td>
            <td>
<span class="badge badge-primary badge-xs">60 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">13731</td>
            <td class="text-xs font-medium">/usr/share/typora/Typora --type=zygote --no-zygote-sandbox</td>
            <td>
<span class="badge badge-primary badge-xs">37 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">11539</td>
            <td class="text-xs font-medium">/usr/bin/nautilus --gapplication-service</td>
            <td>
<span class="badge badge-primary badge-xs">25 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">58239</td>
            <td class="text-xs font-medium">/usr/bin/gnome-text-editor /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-sysmon/nbs/core/html_ids.ipynb</td>
            <td>
<span class="badge badge-primary badge-xs">15 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">200711</td>
            <td class="text-xs font-medium">/usr/bin/gnome-control-center</td>
            <td>
<span class="badge badge-primary badge-xs">15 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
          <tr>
            <td class="text-xs text-base-content">275004</td>
            <td class="text-xs font-medium">/usr/bin/nvidia-settings</td>
            <td>
<span class="badge badge-primary badge-xs">0 MB</span>            </td>
            <td class="text-xs text-success">0%</td>
            <td class="text-xs text-base-content">GPU 0</td>
          </tr>
        </tbody>
      </table>
    </div>
  </div>
</div>

```

In [ ]:
#| export
def render_temperature_sensors(
    temp_info:list  # List of dictionaries containing temperature information
)-> FT:  # A Div element containing temperature sensors
    """Render just the temperature sensors section."""
    # Group temperatures by type
    grouped_temps = {}
    for temp in temp_info:
        temp_type = temp['type']
        if temp_type not in grouped_temps:
            grouped_temps[temp_type] = []
        grouped_temps[temp_type].append(temp)

    return Div(
        *[Div(
            # Sensor type header
            P(temp_type.replace('_', ' ').title(),
              cls=combine_classes(font_size.sm, font_weight.semibold, m.b(2), text_dui.base_content)),

            # Individual sensors
            Div(
                *[Div(
                    Label(
                        Span(sensor['label'], cls=combine_classes(font_size.xs, text_dui.base_content)),
                        Label(
                            Span(
                                f"{sensor['current']:.1f}°C",
                                cls=combine_classes(
                                    font_weight.medium,
                                    get_temperature_color(
                                        sensor['current'],
                                        sensor['high'] or 85,
                                        sensor['critical'] or 95
                                    )
                                )
                            ),
                            cls=combine_classes(flex_display, items.center)
                        ),
                        cls=combine_classes(flex_display, justify.between, items.center)
                    ),
                    cls=combine_classes(p(2), bg_dui.base_200, rounded.md, m.b(2))
                ) for sensor in sensors],
                cls=""
            ),
            cls=str(m.b(3))
        ) for temp_type, sensors in grouped_temps.items()],
        cls="",
        id=HtmlIds.TEMPERATURE_SENSORS
    )

In [ ]:
#| export
def render_temperature_card(
    temp_info:list  # List of dictionaries containing temperature sensor information
)-> FT:  # A Div element containing the temperature sensors card
    """Render the temperature sensors card using helper functions."""
    if not temp_info:
        return Div(
            Div(
                H3("Temperature Sensors", cls=combine_classes(card_title, text_dui.base_content)),
                cls=str(m.b(4))
            ),
            Div(
                "No temperature sensors detected",
                cls=combine_classes(alert, alert_colors.info)
            ),
            cls=str(card_body)
        )

    # Find the highest temperature for the header badge
    max_temp = max((t['current'] for t in temp_info), default=0)

    return Div(
        Div(
            H3("Temperature Sensors", cls=combine_classes(card_title, text_dui.base_content)),
            Span(
                f"{max_temp:.1f}°C",
                cls=combine_classes(
                    badge,
                    get_temperature_badge_color(max_temp),
                    badge_sizes.lg
                )
            ),
            cls=combine_classes(flex_display, justify.between, items.center, m.b(4))
        ),

        # Temperature sensors - render using helper
        render_temperature_sensors(temp_info),

        cls=str(card_body),
        id=HtmlIds.TEMPERATURE_CARD_BODY
    )

In [ ]:
from cjm_fasthtml_sysmon.monitors.sensors import get_temperature_info

temp_info = get_temperature_info()
render_temperature_card(temp_info=temp_info)

```html
<div id="temperature-card-body" class="card-body">
  <div class="flex justify-between items-center mb-4">
    <h3 class="card-title text-base-content">Temperature Sensors</h3>
<span class="badge badge-primary badge-lg">63.9°C</span>  </div>
  <div id="temperature-sensors">
    <div class="mb-3">
      <p class="text-sm font-semibold mb-2 text-base-content">R8169 0 700:00</p>
      <div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">r8169_0_700:00</span><label class="flex items-center"><span class="font-medium text-primary">51.0°C</span></label></label>        </div>
      </div>
    </div>
    <div class="mb-3">
      <p class="text-sm font-semibold mb-2 text-base-content">Nvme</p>
      <div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Composite</span><label class="flex items-center"><span class="font-medium text-primary">51.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 1</span><label class="flex items-center"><span class="font-medium text-primary">51.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 2</span><label class="flex items-center"><span class="font-medium text-primary">61.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Composite</span><label class="flex items-center"><span class="font-medium text-primary">50.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 1</span><label class="flex items-center"><span class="font-medium text-primary">50.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 2</span><label class="flex items-center"><span class="font-medium text-primary">63.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Composite</span><label class="flex items-center"><span class="font-medium text-success">49.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 1</span><label class="flex items-center"><span class="font-medium text-primary">53.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 2</span><label class="flex items-center"><span class="font-medium text-success">48.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 3</span><label class="flex items-center"><span class="font-medium text-success">49.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Composite</span><label class="flex items-center"><span class="font-medium text-success">48.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 1</span><label class="flex items-center"><span class="font-medium text-primary">56.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 2</span><label class="flex items-center"><span class="font-medium text-success">48.9°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Sensor 3</span><label class="flex items-center"><span class="font-medium text-success">47.9°C</span></label></label>        </div>
      </div>
    </div>
    <div class="mb-3">
      <p class="text-sm font-semibold mb-2 text-base-content">R8169 0 800:00</p>
      <div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">r8169_0_800:00</span><label class="flex items-center"><span class="font-medium text-primary">51.0°C</span></label></label>        </div>
      </div>
    </div>
    <div class="mb-3">
      <p class="text-sm font-semibold mb-2 text-base-content">K10Temp</p>
      <div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">Tctl</span><label class="flex items-center"><span class="font-medium text-primary">58.9°C</span></label></label>        </div>
      </div>
    </div>
    <div class="mb-3">
      <p class="text-sm font-semibold mb-2 text-base-content">Spd5118</p>
      <div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">spd5118</span><label class="flex items-center"><span class="font-medium text-success">43.8°C</span></label></label>        </div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">spd5118</span><label class="flex items-center"><span class="font-medium text-success">42.8°C</span></label></label>        </div>
      </div>
    </div>
    <div class="mb-3">
      <p class="text-sm font-semibold mb-2 text-base-content">Amdgpu</p>
      <div>
        <div class="p-2 bg-base-200 rounded-md mb-2">
<label class="flex justify-between items-center"><span class="text-xs text-base-content">edge</span><label class="flex items-center"><span class="font-medium text-success">47.0°C</span></label></label>        </div>
      </div>
    </div>
  </div>
</div>

```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()